In [234]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install rq

In [235]:
# Import packages for use
import pandas as pd
import requests as rq
from io import BytesIO
import re

# importing .xlsx files from github
sheets = [1,2,3,4,5,6,7,8,9,10,11,12,13] # each county one sheet

data_rq1 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/raw/e79f6b7bdcdc20c12a65c19373776eb25000edae/GR14_Bewerberdatei_SN_KS_LK_Gem.xlsx').content
data_rq2 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/raw/e79f6b7bdcdc20c12a65c19373776eb25000edae/GR19_Bewerber_Stand_Juli2020_Final.xlsx').content

sheets_dict = pd.read_excel(BytesIO(data_rq1), sheet_name=sheets, header=2)
sheets_dict_2 = pd.read_excel(BytesIO(data_rq2), sheet_name=sheets, header=2)

In [236]:
# get append datasets
data_appended_1 = pd.DataFrame()
data_appended_2 = pd.DataFrame()
for name, sheet in sheets_dict.items():
    data_appended_1 = data_appended_1.append(sheet)
    data_appended_1['year'] = 2014
for name, sheet in sheets_dict_2.items():
    data_appended_2 = data_appended_2.append(sheet)
    data_appended_2['year'] = 2019
data_appended = data_appended_1.append(data_appended_2)
data_appended.reset_index(inplace=True, drop=True)
data_appended

,BZ_BZID,EBENE,ORTNR,ORTNAME,EBENE 1,ORTNR 1,ORTNAME 1,KW_LISNR,Kurzbezeichnung,Langbezeichnung,Anzahl Bewerber,darunter Frauen,year,Unnamed: 12
0,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",1,DIE LINKE,DIE LINKE,40,11,2014,NaN
1,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",2,CDU,Christlich Demokratische Union Deutschlands,49,11,2014,NaN
2,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",3,SPD,Sozialdemokratische Partei Deutschlands,63,19,2014,NaN
3,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",4,FDP,Freie Demokratische Partei,78,18,2014,NaN
4,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",5,GRÜNE,BÜNDNIS 90/DIE GRÜNEN,64,26,2014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4962,GR19,LK,14730,Nordsachsen,GE,14730340,Wiedemar,2,SPD,Sozialdemokratische Partei Deutschlands,7,2,2019,NaN
4963,GR19,LK,14730,Nordsachsen,GE,14730340,Wiedemar,3,Freie Wähler,Freie Wählervereinigung Wiedemar,14,6,2019,NaN
4964,GR19,LK,14730,Nordsachsen,GE,14730340,Wiedemar,4,Sächsicher Nordzipfel,"Heimatverein ""Sächsicher Nordzipfel""",4,3,2019,NaN
4965,GR19,LK,14730,Nordsachsen,GE,14730360,Zschepplin,1,CDU,Christlich Demokratische Union Deutschlands,14,3,2019,NaN


In [237]:
# rename columns and filter
data_appended = data_appended[['ORTNAME 1', 'Kurzbezeichnung', 'Anzahl Bewerber', 'darunter Frauen', 'year']]
data_appended.rename({'ORTNAME 1':'city', 'Kurzbezeichnung': 'party'}, axis=1, inplace=True)
data_appended['state'] = 'SN'

data_appended = data_appended[(data_appended['party'] == 'DIE LINKE') | (data_appended['party'] == 'CDU') | (data_appended['party'] == 'GRÜNE')| (data_appended['party'] == 'SPD')  | (data_appended['party'] == 'FDP') | (data_appended['party'] == 'AfD')]
data_appended['Prozent Frauen'] = data_appended['darunter Frauen']/data_appended['Anzahl Bewerber'] * 100
data_appended.head()

C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,city,party,Anzahl Bewerber,darunter Frauen,year,state,Prozent Frauen
0,"Chemnitz, Stadt",DIE LINKE,40,11,2014,SN,27.500000
1,"Chemnitz, Stadt",CDU,49,11,2014,SN,22.448980
2,"Chemnitz, Stadt",SPD,63,19,2014,SN,30.158730
3,"Chemnitz, Stadt",FDP,78,18,2014,SN,23.076923
4,"Chemnitz, Stadt",GRÜNE,64,26,2014,SN,40.625000


In [238]:
data_appended = data_appended[~data_appended['city'].str.contains('Wahlkreis')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Leipzig$')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Bautzen$')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Meißen$')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Görlitz$')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Zwickau$')]

In [239]:
# import BW data
data_rq3 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/blob/main/GW2014_Gemeindetabelle_Bewerber_Gew%C3%A4hlte_nach_Geschlecht.xlsx?raw=true').content
data_rq4 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/blob/main/GW2019_Gemeindetabelle_Bewerbungen01.xlsx?raw=true').content
data_rq5 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/blob/main/GW2019_Gemeindetabelle_Bewerbungen02.xlsx?raw=true').content

data_bw_2014 = pd.read_excel(BytesIO(data_rq3), header=4)
data_bw_2019_1 = pd.read_excel(BytesIO(data_rq4), header=2)
data_bw_2019_2 = pd.read_excel(BytesIO(data_rq5), header=2)

In [240]:
# get data into long format
data_bw_2014.rename({'Unnamed: 1':'Key', 'Unnamed: 2':'city', 'Unnamed: 3':'category', 'Unnamed: 4': 'unit'},axis=1, inplace=True)
data_bw_2014 = data_bw_2014[['Key', 'city', 'category', 'unit', 'CDU', 'SPD', 'FDP', 'GRÜNE', 'AfD', 'DIE LINKE']]
data_bw_2014.dropna(inplace=True)
data_long = data_bw_2014[['Key', 'city', 'category', 'unit', 'CDU']]
data_long['party'] = 'CDU'
data_long.rename({'CDU':'value'},axis=1, inplace=True)

list_cols = ['Key', 'city', 'category', 'unit']
for party in ('SPD', 'FDP', 'GRÜNE', 'AfD', 'DIE LINKE'):
    list_cols.append(party)
    data_temp = data_bw_2014[list_cols]
    data_temp['party'] = party
    data_temp.rename({party:'value'},axis=1, inplace=True)
    list_cols = ['Key', 'city', 'category', 'unit']
    data_long = data_long.append(data_temp)
    
del data_temp
data_long.reset_index(drop=True, inplace=True)

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [241]:
# adjust format to match above format
data_long = data_long[(data_long['category'] != 'Gewählte') & (data_long['category'] != 'Gewählte Frauen')]
data_long['type'] = 'Anzahl Bewerber'
data_long.loc[(data_long['category'] == 'Bewerberinnen') & (data_long['unit'] == 'Anzahl'), 'type'] = 'darunter Frauen'
data_long.loc[(data_long['category'] == 'Bewerberinnen') & (data_long['unit'] == '%'), 'type'] = 'Prozent Frauen'
data_long.drop(['category', 'unit'], axis=1)
data_long_2014 = data_long.pivot(index=['Key', 'city', 'party'], columns=['type'], values='value')
data_long_2014['year'] = 2014
data_long_2014.reset_index(inplace=True)
data_long_2014['state'] = 'BW'

In [242]:
# renaming and filtering
data_bw_2019_1.rename({'Davon':'CDU', 'Unnamed: 8':'GRÜNE', 'Unnamed: 9':'SPD', 'Unnamed: 10':'FDP', 'Unnamed: 11':'AfD'}, axis=1, inplace=True)
data_bw_2019_1 = data_bw_2019_1[['Schlüssel-\nnummer', 'Gemeinde', 'Bezeichnung', 'Ein-\nheit', 'CDU', 'SPD', 'FDP', 'GRÜNE', 'AfD']]
data_bw_2019_1.dropna(inplace=True)
data_bw_2019_2.rename({'Unnamed: 9':'DIE LINKE'}, axis=1, inplace=True)
data_bw_2019_2 = data_bw_2019_2[['Schlüssel-\nnummer', 'Gemeinde', 'Bezeichnung', 'Ein-\nheit', 'DIE LINKE']]
data_bw_2019_2.dropna(inplace=True)
data_bw_2019 = pd.merge(data_bw_2019_1, data_bw_2019_2, on=['Schlüssel-\nnummer', 'Gemeinde', 'Bezeichnung', 'Ein-\nheit'], how='inner', validate='1:1')
data_bw_2019

,Schlüssel-\nnummer,Gemeinde,Bezeichnung,Ein-\nheit,CDU,SPD,FDP,GRÜNE,AfD,DIE LINKE
0,111,"Stuttgart, Landeshauptstadt",Bewerberinnen und Bewerber,Anzahl,60,60,60,60,60,60
1,111,"Stuttgart, Landeshauptstadt",Bewerberinnen,Anzahl,25,30,18,30,16,29
2,111,"Stuttgart, Landeshauptstadt",Bewerberinnenanteil,%,41.666667,50,30,50,26.666667,48.333333
3,111,"Stuttgart, Landeshauptstadt",Unionsbürger,Anzahl,1,0,2,4,0,0
4,111,"Stuttgart, Landeshauptstadt",Unionsbürgeranteil,%,1.666667,0,3.333333,6.666667,0,0
...,...,...,...,...,...,...,...,...,...,...
5500,437,Herdwangen-Schönach,Bewerberinnen und Bewerber,Anzahl,7,0,0,0,0,0
5501,437,Herdwangen-Schönach,Bewerberinnen,Anzahl,0,0,0,0,0,0
5502,437,Herdwangen-Schönach,Bewerberinnenanteil,%,0,0,0,0,0,0
5503,437,Herdwangen-Schönach,Unionsbürger,Anzahl,0,0,0,0,0,0


In [243]:
# same procedure as above for 2014 data
data_bw_2019 = data_bw_2019[(data_bw_2019['Bezeichnung'] != 'Unionsbürger') & (data_bw_2019['Bezeichnung'] != 'Unionsbürgeranteil')]
data_bw_2019.rename({'Gemeinde': 'city', 'Schlüssel-\nnummer': 'Key', 'Bezeichnung':'category', 'Ein-\nheit':'unit'}, axis=1, inplace=True)
data_long = data_bw_2019[['Key', 'city', 'category', 'unit', 'CDU']]
data_long['party'] = 'CDU'
data_long.rename({'CDU':'value'},axis=1, inplace=True)

list_cols = ['Key', 'city', 'category', 'unit']
for party in ('SPD', 'FDP', 'GRÜNE', 'AfD', 'DIE LINKE'):
    list_cols.append(party)
    data_temp = data_bw_2019[list_cols]
    data_temp['party'] = party
    data_temp.rename({party:'value'},axis=1, inplace=True)
    list_cols = ['Key', 'city', 'category', 'unit']
    data_long = data_long.append(data_temp)
    
del data_temp
data_long.reset_index(drop=True, inplace=True)

C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [244]:
# filter and organize data
data_long['type'] = 'Anzahl Bewerber'
data_long.loc[(data_long['category'] == 'Bewerberinnen') & (data_long['unit'] == 'Anzahl'), 'type'] = 'darunter Frauen'
data_long.loc[(data_long['category'] == 'Bewerberinnenanteil') & (data_long['unit'] == '%'), 'type'] = 'Prozent Frauen'
data_long.drop(['category', 'unit'], axis=1)
data_long_2019 = data_long.pivot(index=['Key', 'city', 'party'], columns=['type'], values='value')
data_long_2019.reset_index(drop=False, inplace=True)
data_long_2019['year'] = 2019

In [245]:
# append bw data
data_bw = data_long_2019.append(data_long_2014)
data_bw.reset_index(drop=True,inplace=True)
data_bw['state'] = 'BW'
data_bw.drop('Key', axis=1, inplace=True)

In [246]:
# merge SN and BW data
data_so_far = data_bw.append(data_appended)
data_so_far.reset_index(drop=True,inplace=True)

In [247]:
data_th_2014 = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/Th_2014.csv', header=3, sep=';')
data_th_2019 = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/Th_2019.csv', header=3, sep=';')
data_th_2014['year'] = 2014
data_th_2019['year'] = 2019
data_th_2014['state'] = 'TH'
data_th_2019['state'] = 'TH'
data_th_2014.dropna(inplace=True)
data_th_2019.dropna(inplace=True)
data_th_2014.rename({'Unnamed: 3': 'darunter Frauen', 'Unnamed: 5': 'female_mem'}, axis=1, inplace=True)
data_th_2019.rename({'Unnamed: 3': 'darunter Frauen', 'Unnamed: 5': 'female_mem'}, axis=1, inplace=True)
data_th = data_th_2019.append(data_th_2014)
data_th['Gemeinde'] = data_th['Gemeinde'].str.split(', ', expand=True)[0]
data_th['plz'] = data_th['Gemeinde'].str.split(r'[\s]', expand=True)[0]
data_th['Gemeinde'] = data_th['Gemeinde'].str.split(r'[[0-9]+[\s]]?', expand=True)[1]
data_th['darunter Frauen'] = data_th['darunter Frauen'].str.replace('-', '0').astype(float)

In [248]:
data_th.dropna(inplace=True)
data_th.reset_index(inplace=True, drop=True)
data_th = data_th[['Gemeinde', 'Wahlvorschlag', 'Anzahl der Bewerber', 'darunter Frauen', 'year', 'state']]
data_th.rename({'Gemeinde':'city', 'Wahlvorschlag':'party', 'Anzahl der Bewerber':'Anzahl Bewerber'}, axis=1, inplace=True)
data_th['Anzahl Bewerber'] = data_th['Anzahl Bewerber'].astype(float)
data_th['Prozent Frauen'] = data_th['darunter Frauen']/data_th['Anzahl Bewerber']*100
data_th = data_th[data_th['party'] != 'Sonstige']
data_th

,city,party,Anzahl Bewerber,darunter Frauen,year,state,Prozent Frauen
0,Erfurt,CDU,49.0,13.0,2019,TH,26.530612
1,Gera,CDU,16.0,3.0,2019,TH,18.750000
2,Jena,CDU,46.0,10.0,2019,TH,21.739130
3,Suhl,CDU,35.0,7.0,2019,TH,20.000000
4,Weimar,CDU,42.0,9.0,2019,TH,21.428571
...,...,...,...,...,...,...,...
2169,Neuhaus am Rennweg,GRÜNE,1.0,1.0,2014,TH,100.000000
2170,Sonneberg,GRÜNE,7.0,1.0,2014,TH,14.285714
2171,Rudolstadt,GRÜNE,10.0,4.0,2014,TH,40.000000
2172,Saalfeld/Saale,GRÜNE,13.0,5.0,2014,TH,38.461538


In [249]:
data_so_far = data_so_far.append(data_th)
data_so_far.reset_index(drop=True,inplace=True)

In [250]:
# Brandenburg Data

file_list_2014 = ['2014 GV - LK Barnim', '2014 GV - LK Dahme-Spreewald', '2014 GV - LK Elbe-Elster', '2014 GV - LK Havelland', '2014 GV - LK Märkisch-Oderland', '2014 GV - LK Oberhavel', '2014 GV - LK Oberspreewald-Lausitz', 
                  '2014 GV - LK Oder-Spree', '2014 GV - LK Ostprignitz-Ruppin', '2014 GV - LK Potsdam-Mittelmark', '2014 GV - LK Prignitz', '2014 GV - LK Spree-Neiße', '2014 GV - LK Teltow-Fläming', '2014 GV - LK Uckermark']

file_2014_2 = r'C:\Users\mariu\Documents\pol_viol\pol_viol\Gemeindevertretungen_2014\2014 KT und SVV.xlsx' # sheets [1:5]

file_list_2019 = ['2019 GV - 60 LK Barnim', '2019 GV - 61 LK Dahme-Spreewald', '2019 GV - 62 LK Elbe-Elster', '2019 GV - 63 LK Havelland', '2019 GV - 64 LK Märkisch-Oderland', '2019 GV - 65 LK Oberhavel', '2019 GV - 66 LK Oberspreewald-Lausitz'
                  , '2019 GV - 67 LK Oder-Spree', '2019 GV - 68 LK Ostprignitz-Ruppin', '2019 GV - 69 LK Potsdam-Mittelmark', '2019 GV - 70 LK Prignitz', '2019 GV - 71 LK Spree-Neiße', '2019 GV - 72 LK Teltow-Fläming', 
                 '2019 GV - 73 LK Uckermark']

file_2019_2 = r'C:\Users\mariu\Documents\pol_viol\pol_viol\Gemeindevertretungen_2019\2019 KT und SVV.xlsx' # sheets [1:5]

In [251]:
# Import county-based data

full_table_14 = pd.DataFrame()
for file in file_list_2014:
    path = r'C:\Users\mariu\Documents\pol_viol\pol_viol\Gemeindevertretungen_2014' + '\\' + file + '.xls'
    
    sheets_dict = pd.read_excel(path, sheet_name=None, skiprows=4)
    for name, sheet in list(sheets_dict.items())[1:]:
        sheet['city'] = name
        full_table_14 = full_table_14.append(sheet)

full_table_19 = pd.DataFrame()
for file in file_list_2019:
    path = r'C:\Users\mariu\Documents\pol_viol\pol_viol\Gemeindevertretungen_2019' + '\\' + file + '.xlsx'
    
    sheets_dict = pd.read_excel(path, sheet_name=None, skiprows=4)
    for name, sheet in list(sheets_dict.items())[1:]:
        sheet['city'] = name
        full_table_19 = full_table_19.append(sheet)
                
full_table_14.reset_index(inplace=True, drop=True)
full_table_14['year'] = 2014
full_table_19.reset_index(inplace=True, drop=True)
full_table_19['year'] = 2019

C:\Users\mariu\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [252]:
full_table_19.dropna(thresh=3,axis=1, inplace=True)
full_table_14.dropna(thresh=3,axis=1, inplace=True)

In [253]:
full_table_14.rename({'Unnamed: 0':'party_long', 'Unnamed: 2':'party', 'Unnamed: 3':'Wg_o_P', 'insgesamt':'Anzahl Bewerber',
                     'dar. weiblich':'darunter Frauen'}, axis=1, inplace=True)
full_table_14.drop(['Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)

In [254]:
full_table_19.rename({'Unnamed: 0':'party_long', 'Unnamed: 2':'party', 'Unnamed: 3':'Wg_o_P', 'insgesamt':'Anzahl Bewerber',
                     'dar. weiblich':'darunter Frauen'}, axis=1, inplace=True)
full_table_19.drop(['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 1'], axis=1, inplace=True)

In [255]:
full_table_bb = full_table_14.append(full_table_19)
full_table_bb['city'] = full_table_bb['city'].str.replace(r'(\s[0-9]*)$', '')

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [256]:
# Stadtkreise 2014
sk_2014 = pd.read_excel(file_2014_2, sheet_name=None, skiprows=4)
file_sk_2014 = pd.DataFrame()
for name, sheet in list(sk_2014.items())[1:5]:
    sheet['city'] = name #rename cities
    file_sk_2014 = file_sk_2014.append(sheet) 
    
file_sk_2014['year'] = 2014

In [257]:
# Stadtkreise 2019
sk_2019 = pd.read_excel(file_2019_2, sheet_name=None, skiprows=4)
file_sk_2019 = pd.DataFrame()
for name, sheet in list(sk_2019.items())[1:5]:
    sheet['city'] = name #rename cities
    file_sk_2019 = file_sk_2019.append(sheet)
    
file_sk_2019['year'] = 2019

In [258]:
file_sk_2014.rename({'Unnamed: 0':'party_long', 'Unnamed: 1':'party', 'Unnamed: 2':'Wg_o_P',
                     'Unnamed: 4':'Anzahl Bewerber', 'weibliche\nBewerbende':'darunter Frauen'}, axis=1, inplace=True)
file_sk_2014.drop('Unnamed: 3', axis=1, inplace=True)
file_sk_2014['city'] = file_sk_2014['city'].str.replace('51 - BRB', 'Brandeburg an der Havel').str.replace('52 - CB', 'Cottbus')
file_sk_2014['city'] = file_sk_2014['city'].str.replace('53 - FF', 'Frankfurt (Oder)').str.replace('54 - P', 'Potsdam')

In [259]:
file_sk_2019.rename({'Unnamed: 0':'party_long', 'Unnamed: 1':'party', 'Unnamed: 2':'Wg_o_P',
                     'Unnamed: 4':'Anzahl Bewerber', 'weibliche\nBewerbende':'darunter Frauen'}, axis=1, inplace=True)
file_sk_2019.drop('Unnamed: 3', axis=1, inplace=True)
file_sk_2019['city'] = file_sk_2019['city'].str.replace('51-BRB', 'Brandeburg an der Havel').str.replace('52-CB', 'Cottbus')
file_sk_2019['city'] = file_sk_2019['city'].str.replace('53-FF', 'Frankfurt (Oder)').str.replace('54-P', 'Potsdam')

In [260]:
file_sk = file_sk_2014.append(file_sk_2019)

In [261]:
full_table_bb = file_sk.append(full_table_bb)

In [262]:
full_table_bb = full_table_bb[full_table_bb['Wg_o_P'] == 'P']
full_table_bb.drop(['Personen mit Unionsbürger-schaft', 'dar. Personen mit Unionsbürgerschaft'], axis=1, inplace=True)
full_table_bb.reset_index(drop=True, inplace=True)
full_table_bb.drop(['Wg_o_P'], axis=1, inplace=True)

In [263]:
full_table_bb['state'] = 'BB'

In [264]:
data_so_far = data_so_far.append(full_table_bb)

In [265]:
data_so_far.drop(['Prozent Frauen', 'party_long'], axis=1, inplace=True)
data_so_far.reset_index(drop=True, inplace=True)
data_so_far

,city,party,Anzahl Bewerber,darunter Frauen,year,state
0,"Stuttgart, Landeshauptstadt",AfD,60,16,2019,BW
1,"Stuttgart, Landeshauptstadt",CDU,60,25,2019,BW
2,"Stuttgart, Landeshauptstadt",DIE LINKE,60,29,2019,BW
3,"Stuttgart, Landeshauptstadt",FDP,60,18,2019,BW
4,"Stuttgart, Landeshauptstadt",GRÜNE,60,30,2019,BW
...,...,...,...,...,...,...
19001,Pinnow,CDU,9,2,2019,BB
19002,Pinnow,DIE LINKE,1,1,2019,BB
19003,Pinnow,GRÜNE/B 90,1,0,2019,BB
19004,Schöneberg,DIE LINKE,1,0,2019,BB


In [270]:
data_so_far['party'] = data_so_far['party'].str.replace('GRÜNE/B 90', 'GRÜNE'
                        )

,city,party,Anzahl Bewerber,darunter Frauen,year,state
0,"Stuttgart, Landeshauptstadt",AfD,60,16,2019,BW
1,"Stuttgart, Landeshauptstadt",CDU,60,25,2019,BW
2,"Stuttgart, Landeshauptstadt",DIE LINKE,60,29,2019,BW
3,"Stuttgart, Landeshauptstadt",FDP,60,18,2019,BW
4,"Stuttgart, Landeshauptstadt",GRÜNE,60,30,2019,BW
...,...,...,...,...,...,...
19001,Pinnow,CDU,9,2,2019,BB
19002,Pinnow,DIE LINKE,1,1,2019,BB
19003,Pinnow,GRÜNE/B 90,1,0,2019,BB
19004,Schöneberg,DIE LINKE,1,0,2019,BB


In [165]:
# standardize city names for merging w/ crime data
data_so_far['city'] = data_so_far['city'].str.split(',',expand=True)[0]
data_so_far['city'] = data_so_far['city'].str.replace('ä', 'a', re.IGNORECASE).str.replace('ö','o', re.IGNORECASE).str.replace('ü', 'u', re.IGNORECASE).str.replace('ß', 'ss', re.IGNORECASE)

In [166]:
#data_so_far.to_csv(r'C:\Users\mariu\Documents\pol_viol\pol_viol\election_data_edited.csv')

In [167]:
# merge control variables here!
# ID for merging: Sachsen "ORTNR 1"; BW data_bw_2014["Key"]; 
# merge plz zu Sachsen und BW
# PLZ: THüringen "plz"
# merge gemeindeschlüssel ID zu THüringen